In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option('display.max_columns', 100)

In [2]:
%%time
df=pd.read_csv("train.csv",index_col="UniqueID")
desc=pd.read_excel("Data Dictionary.xlsx")
test = pd.read_csv('test.csv', index_col="UniqueID")
#desc

Wall time: 17.8 s


Changing DisbursalDate and Data.of.Birth column to a Datetime type

In [14]:
%%time
#a=pd.to_datetime(df['DisbursalDate'])
dd=pd.to_datetime(df['DisbursalDate']).dt.strftime('%d/%m/%Y')
dob=pd.to_datetime(df['Date.of.Birth']).dt.strftime('%d/%m/%Y')

Wall time: 6min 34s


Creating Age column from disbursal date (dd) and dateofBirth (dob)

In [15]:
%%time
df['Age']=pd.to_datetime(dd).dt.year - pd.to_datetime(dob).dt.year

Wall time: 6min 12s


In [16]:
df['Age'] = abs(df['Age'])

Dropping the negative rows of Age in the dataframe because we can not have a negative value for age

In [17]:
df.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,Age
UniqueID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,03-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0,34
537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,26-09-18,6,1998,1,1,0,0,0,0,598,I-Medium Risk,1,1,1,27600,50200,50200,0,0,0,0,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1,33
417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,01-08-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0,33
624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,26-10-18,6,1998,1,1,0,0,0,0,305,L-Very High Risk,3,0,0,0,0,0,0,0,0,0,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1,25
539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,26-09-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1,41


Extracting the digits in AVERAGE.ACCT.AGE and AVERAGE.ACCT.AGE columns

In [18]:
a=df['AVERAGE.ACCT.AGE'].str.extractall(r'(\d+)').unstack()

# multiply first elements by 12 to convert to months, then add the second element to it 
df['AVERAGE.ACCT.AGE']=12*pd.to_numeric(a[0][0])+  pd.to_numeric(a[0][1])


In [19]:
a=df['CREDIT.HISTORY.LENGTH'].str.extractall(r'(\d+)').unstack()

# multiply first elements by 12 to convert to months, then add the second element to it 
df['CREDIT.HISTORY.LENGTH']=12*pd.to_numeric(a[0][0])+  pd.to_numeric(a[0][1])

In [20]:
cols_to_drop=['NO.OF_INQUIRIES','Employee_code_ID','Current_pincode_ID',
              'State_ID','manufacturer_id','supplier_id','branch_id','DisbursalDate','Date.of.Birth']
df=df.drop(cols_to_drop,axis=1)

#df=df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233154 entries, 420825 to 630213
Data columns (total 32 columns):
disbursed_amount                       233154 non-null int64
asset_cost                             233154 non-null int64
ltv                                    233154 non-null float64
Employment.Type                        225493 non-null object
MobileNo_Avl_Flag                      233154 non-null int64
Aadhar_flag                            233154 non-null int64
PAN_flag                               233154 non-null int64
VoterID_flag                           233154 non-null int64
Driving_flag                           233154 non-null int64
Passport_flag                          233154 non-null int64
PERFORM_CNS.SCORE                      233154 non-null int64
PERFORM_CNS.SCORE.DESCRIPTION          233154 non-null object
PRI.NO.OF.ACCTS                        233154 non-null int64
PRI.ACTIVE.ACCTS                       233154 non-null int64
PRI.OVERDUE.ACCTS         

In [21]:
cat_col=['Employment.Type', 'MobileNo_Avl_Flag', 
         'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 
         'Driving_flag', 'Passport_flag','loan_default']
# changing the type of selected columns in cat_col
for item in cat_col:
    df[item]=df[item].astype('category')

In [22]:
df=df.drop('PERFORM_CNS.SCORE.DESCRIPTION',axis=1)

In [23]:
# filling missing rows in Employment.Type column with its mode 
df['Employment.Type']=df['Employment.Type'].fillna(df['Employment.Type'].mode().iloc[0])

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233154 entries, 420825 to 630213
Data columns (total 31 columns):
disbursed_amount                       233154 non-null int64
asset_cost                             233154 non-null int64
ltv                                    233154 non-null float64
Employment.Type                        233154 non-null category
MobileNo_Avl_Flag                      233154 non-null category
Aadhar_flag                            233154 non-null category
PAN_flag                               233154 non-null category
VoterID_flag                           233154 non-null category
Driving_flag                           233154 non-null category
Passport_flag                          233154 non-null category
PERFORM_CNS.SCORE                      233154 non-null int64
PRI.NO.OF.ACCTS                        233154 non-null int64
PRI.ACTIVE.ACCTS                       233154 non-null int64
PRI.OVERDUE.ACCTS                      233154 non-null int64
PRI.CUR

In [25]:
#Import the resample class from sklearn to balance the data for the target column
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df.loan_default==0]
df_minority = df[df.loan_default==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=182543, # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df.loan_default.value_counts()
#df.columns

1    182543
0    182543
Name: loan_default, dtype: int64

In [26]:
cat_new=['Employment.Type', 'MobileNo_Avl_Flag', 
         'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 
         'Driving_flag', 'Passport_flag']
# using one hot encoding 
df_cat=pd.get_dummies(df[cat_new], drop_first=True)

In [27]:
df_cat.head()

,Employment.Type_Self employed,Aadhar_flag_1,PAN_flag_1,VoterID_flag_1,Driving_flag_1,Passport_flag_1
UniqueID,,,,,,
420825,0,1,0,0,0,0
417566,1,1,0,0,0,0
518279,1,1,0,0,0,0
529269,0,1,0,0,0,0
510278,0,1,0,0,0,0


In [28]:
# dropping the cat cols
df_new=df.drop(cat_new,axis=1)

In [29]:
# joining with the new cat cols
df=df_new.join(df_cat)

In [24]:
df.head()
df.to_csv('clean_train')

In [21]:
X = df.drop('loan_default', axis=1)
y = df['loan_default']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(818318, 29)
(204580, 29)
(818318,)
(204580,)


In [25]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.pipeline import Pipeline


# First create the baseline model to tune
rf = RandomForestClassifier()
knn = KNeighborsClassifier(n_neighbors=)

# Random search of parameters, using 3 fold cross validation, 
# search across 15 different combinations, and use all available cores
#rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, cv = 3, verbose=2, random_state=42, n_jobs = 1)

# Fit the random search model
# rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

MemoryError: 

In [ ]:
%%time
#Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import classification_report, roc_curve, accuracy_score, roc_auc_score
y_pred = rf.predict(X_test)
#y_proba_test = best_model.predict_proba(test_features)
# Model Accuracy, how often is the classifier correct?
print("The model accuracy on the test data is: %.2f%%" %(accuracy_score(y_test, y_pred)*100))
#print("The precision score on the test data is: %.2f" %(precision_score(test_labels, y_pred)))
#print("The recall score on the test data is: %.2f" %(recall_score(train_labels, y_pred,average='binary')))
print(classification_report(y_test,y_pred))
print("The model roc_auc score on the test data is: %.2f%%" %(roc_auc_score(y_test, y_pred)))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=8)
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)
#y_proba_test = best_model.predict_proba(test_features)
# Model Accuracy, how often is the classifier correct?
print("The model accuracy on the test data is: %.2f%%" %(accuracy_score(y_test, y_pred)*100))
#print("The precision score on the test data is: %.2f" %(precision_score(test_labels, y_pred)))
#print("The recall score on the test data is: %.2f" %(recall_score(train_labels, y_pred,average='binary')))
print(classification_report(y_test,y_pred))
print("The model roc_auc score on the test data is: %.2f%%" %(roc_auc_score(y_test, y_pred)))

In [ ]:
fpr, tpr, threshold = roc_curve(y_test, y_pred)
plt.plot([0,1], [0,1])
plt.plot(fpr, tpr, label='Ramdom Forest')
plt.xlabel('False Positive')
plt.ylabel('True Positive')
plt.title('ROC CURVE')
plt.show()

PreProcessing the test Data

In [3]:
test.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
UniqueID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
655269,53478,63558,86.54,67,22807,45,1497,01-01-74,Salaried,03-11-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
723482,55513,63163,89.45,67,22807,45,1497,20-05-85,Self employed,20-11-18,6,1998,1,1,0,0,0,0,749,C-Very Low Risk,2,1,0,43898,48780,48780,0,0,0,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1
758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,29-11-18,4,1646,1,1,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
763449,46905,63896,76.58,78,17014,45,2070,01-06-73,Self employed,29-11-18,4,1646,1,1,0,0,0,0,14,Not Scored: Only a Guarantor,1,1,1,132480,255000,255000,0,0,0,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0
708663,51428,63896,86.08,78,17014,45,2069,01-06-72,Salaried,17-11-18,4,1646,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0


In [4]:
%%time
dd=pd.to_datetime(test['DisbursalDate']).dt.strftime('%d/%m/%Y')
dob=pd.to_datetime(test['Date.of.Birth']).dt.strftime('%d/%m/%Y')

Wall time: 3min 41s


In [5]:
%%time
test['Age']=pd.to_datetime(dd).dt.year - pd.to_datetime(dob).dt.year

Wall time: 3min 20s


In [6]:
test['Age'] = abs(test['Age'])
test.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,DisbursalDate,State_ID,Employee_code_ID,MobileNo_Avl_Flag,Aadhar_flag,PAN_flag,VoterID_flag,Driving_flag,Passport_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,Age
UniqueID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
655269,53478,63558,86.54,67,22807,45,1497,01-01-74,Salaried,03-11-18,6,1998,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,44
723482,55513,63163,89.45,67,22807,45,1497,20-05-85,Self employed,20-11-18,6,1998,1,1,0,0,0,0,749,C-Very Low Risk,2,1,0,43898,48780,48780,0,0,0,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1,33
758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,29-11-18,4,1646,1,1,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,23
763449,46905,63896,76.58,78,17014,45,2070,01-06-73,Self employed,29-11-18,4,1646,1,1,0,0,0,0,14,Not Scored: Only a Guarantor,1,1,1,132480,255000,255000,0,0,0,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0,45
708663,51428,63896,86.08,78,17014,45,2069,01-06-72,Salaried,17-11-18,4,1646,1,1,0,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,46


In [7]:
a=test['AVERAGE.ACCT.AGE'].str.extractall(r'(\d+)').unstack()

# multiply first elements by 12 to convert to months, then add the second element to it 
test['AVERAGE.ACCT.AGE']=12*pd.to_numeric(a[0][0])+  pd.to_numeric(a[0][1])

b=test['CREDIT.HISTORY.LENGTH'].str.extractall(r'(\d+)').unstack()

# multiply first elements by 12 to convert to months, then add the second element to it 
test['CREDIT.HISTORY.LENGTH']=12*pd.to_numeric(b[0][0])+  pd.to_numeric(b[0][1])

In [8]:
cols_to_drop=['NO.OF_INQUIRIES','Employee_code_ID','Current_pincode_ID',
              'State_ID','manufacturer_id','supplier_id','branch_id','DisbursalDate','Date.of.Birth']
test=test.drop(cols_to_drop,axis=1)

#df=df.reset_index()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112392 entries, 655269 to 755244
Data columns (total 31 columns):
disbursed_amount                       112392 non-null int64
asset_cost                             112392 non-null int64
ltv                                    112392 non-null float64
Employment.Type                        108949 non-null object
MobileNo_Avl_Flag                      112392 non-null int64
Aadhar_flag                            112392 non-null int64
PAN_flag                               112392 non-null int64
VoterID_flag                           112392 non-null int64
Driving_flag                           112392 non-null int64
Passport_flag                          112392 non-null int64
PERFORM_CNS.SCORE                      112392 non-null int64
PERFORM_CNS.SCORE.DESCRIPTION          112392 non-null object
PRI.NO.OF.ACCTS                        112392 non-null int64
PRI.ACTIVE.ACCTS                       112392 non-null int64
PRI.OVERDUE.ACCTS         

In [9]:
cat_col=['Employment.Type', 'MobileNo_Avl_Flag', 
         'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 
         'Driving_flag', 'Passport_flag']
# changing the type of selected columns in cat_col
for item in cat_col:
    test[item]=test[item].astype('category')

In [10]:
test=test.drop('PERFORM_CNS.SCORE.DESCRIPTION',axis=1)
test['Employment.Type']=test['Employment.Type'].fillna(test['Employment.Type'].mode().iloc[0])

In [ ]:
test.info()

In [11]:
cat_new=['Employment.Type', 'MobileNo_Avl_Flag', 
         'Aadhar_flag', 'PAN_flag', 'VoterID_flag', 
         'Driving_flag', 'Passport_flag']
# using one hot encoding 
test_cat=pd.get_dummies(test[cat_new])
test_cat.head()

,Employment.Type_Salaried,Employment.Type_Self employed,MobileNo_Avl_Flag_1,Aadhar_flag_0,Aadhar_flag_1,PAN_flag_0,PAN_flag_1,VoterID_flag_0,VoterID_flag_1,Driving_flag_0,Driving_flag_1,Passport_flag_0,Passport_flag_1
UniqueID,,,,,,,,,,,,,
655269,1,0,1,0,1,1,0,1,0,1,0,1,0
723482,0,1,1,0,1,1,0,1,0,1,0,1,0
758529,1,0,1,0,1,0,1,1,0,1,0,1,0
763449,0,1,1,0,1,1,0,1,0,1,0,1,0
708663,1,0,1,0,1,1,0,1,0,1,0,1,0


In [12]:
test_new=test.drop(cat_new,axis=1)
test=test_new.join(test_cat)
test.head()

,disbursed_amount,asset_cost,ltv,PERFORM_CNS.SCORE,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,Age,Employment.Type_Salaried,Employment.Type_Self employed,MobileNo_Avl_Flag_1,Aadhar_flag_0,Aadhar_flag_1,PAN_flag_0,PAN_flag_1,VoterID_flag_0,VoterID_flag_1,Driving_flag_0,Driving_flag_1,Passport_flag_0,Passport_flag_1
UniqueID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
655269,53478,63558,86.54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,44,1,0,1,0,1,1,0,1,0,1,0,1,0
723482,55513,63163,89.45,749,2,1,0,43898,48780,48780,0,0,0,0,0,0,5605,0,1,0,8,12,33,0,1,1,0,1,1,0,1,0,1,0,1,0
758529,65282,84320,79.93,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,1,0,1,0,1,0,1,1,0,1,0,1,0
763449,46905,63896,76.58,14,1,1,1,132480,255000,255000,0,0,0,0,0,0,0,0,0,0,29,29,45,0,1,1,0,1,1,0,1,0,1,0,1,0
708663,51428,63896,86.08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,1,0,1,0,1,1,0,1,0,1,0,1,0


In [13]:
test.shape
test.to_csv('cleaned_test.csv')

In [ ]:
%%time
# predicting
test_pred = rf.predict(test)

In [ ]:
loan_default = pd.DataFrame({'UniqueID': test.index, 'loan_default': test_pred}).to_csv('loan_default.csv')